In [1]:
import volume_predictor as vp
import demand_generator as dg
import numpy as np
import pandas as pd
import random

In [2]:
#Train Model if necessary or want to update the model

csv_file_path = 'Inbound_predict_data.csv'
model_file_path = 'trained_model.pkl'
vp.train_and_save_model(csv_file_path, model_file_path)


Package Count Prediction MAE: 1221.1454545454546
Package Count R2: 0.7727269836067745
Model saved to trained_model.pkl


In [3]:

#Predict Volume

# Example usage
model_file_path = 'trained_model.pkl'
model = vp.load_model(model_file_path)

# Example feature values
feature_values = {
    'yesterday_total_packages': 16152,
    'RAFT_known_shipped_pkg_count': 5264,
    'RAFT_predicted_carryover_pkg_count': 28,
    'RAFT_predicted_total_handoff_pkg_count': 6594,
    'Day of Week_Sunday': 0,
    'Day of Week_Monday': 1,
    'Day of Week_Tuesday': 0,
    'Day of Week_Wednesday': 0,
    'Day of Week_Thursday': 0,
    'Day of Week_Friday': 0,
    'Day of Week_Saturday': 0,
    'Promotion': 0,
    'TMAX': 36.2,
    'TMIN': 31.8,
    'AWND': 9.87,
    'PRCP': 0.912,
    'SNOW': 6.14
}

predicted_volume = vp.predict_volume(model, feature_values)
print(f'Predicted Total Volume: {predicted_volume}')


Predicted Total Volume: 15706


In [4]:
df_package_distribution, TFC_vol, TFC_arrival_minutes = dg.generate_demand('linehaul_all_predict - Copy.csv', 3858, predicted_volume, '2024-09-01')

c:\Users\fenst\Documents\Python\Practice\demand_generator.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['planned_arrival_datetime'] = pd.to_datetime(filtered_df['planned_arrival_datetime'])
c:\Users\fenst\Documents\Python\Practice\demand_generator.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['actual_arrival_datetime'] = pd.to_datetime(filtered_df['actual_arrival_datetime'])


In [5]:
#df_package_distribution.drop(columns=['vol_actualization'], inplace=True)
csv_file = 'carrier_breakdown.csv'
distributions = pd.read_csv(csv_file)

total_packages = predicted_volume
df_pallet_formation = pd.DataFrame(df_package_distribution[['Truck Number','predicted_truck_volume', 'pallets']])

# Determine the number of packages going to each organization based on the distribution
carrier_packages = {}
for index, row in distributions.iterrows():
    while True:
        value = int(np.random.normal(row["average_percent"], row["std_dev"]) * total_packages)
        if value >= 0:
            carrier_packages[row["carrier"]] = value
            break

# Adjust the total to match the exact number of packages
total_assigned_packages = sum(carrier_packages.values())
if total_assigned_packages != total_packages:
    difference = total_packages - total_assigned_packages
    carrier_packages[random.choice(list(carrier_packages.keys()))] += difference
    carrier_packages['TLMD'] = carrier_packages['TLMD'] - TFC_vol

df_carrier_breakdown = pd.DataFrame(list(carrier_packages.items()), columns=['Organization', 'Packages'])


def assign_packages_to_pallets(trucks_df, packages_df):
    result = []
    
    # Create a list of all packages
    all_packages = []
    for j in range(len(packages_df)):
        org = packages_df.loc[j, 'Organization']
        num_packages = packages_df.loc[j, 'Packages']
        all_packages.extend([org] * num_packages)
    
    # Shuffle the list of all packages
    np.random.shuffle(all_packages)
    
    start_index = 0
    for i in range(len(trucks_df)):
        truck_number = trucks_df.loc[i, 'Truck Number']
        num_pallets = trucks_df.loc[i, 'pallets']
        predicted_truck_volume = trucks_df.loc[i, 'predicted_truck_volume']
        
        # Skip trucks with zero pallets
        if num_pallets <= 0:
            continue
        
        # Get the packages for the current truck
        truck_packages = all_packages[start_index:start_index + predicted_truck_volume]
        start_index += predicted_truck_volume
        
        # Create a list of pallets for the current truck
        truck_pallets = [[] for _ in range(num_pallets)]
        
        # Randomly assign packages to pallets on the current truck
        for package in truck_packages:
            pallet_index = np.random.randint(0, num_pallets)
            truck_pallets[pallet_index].append(package)
        
        # Count the number of packages per organization on each pallet
        pallet_counts = []
        for pallet in truck_pallets:
            counts = {org: pallet.count(org) for org in packages_df['Organization']}
            pallet_counts.append(counts)
        
        result.append({
            'Truck Number': truck_number,
            'pallets': pallet_counts
        })
    
    return result


assigned_packages = assign_packages_to_pallets(df_pallet_formation, df_carrier_breakdown)
print(assigned_packages)
# for truck in assigned_packages:
#     print(f"Truck {truck['Truck Number']}:")
#     for i, pallet in enumerate(truck['pallets']):
#         print(f"  Pallet {i+1}: {pallet}")


[{'Truck Number': 1, 'pallets': [{'TLMD': 30, 'FDEG': 6, 'UPSN': 6, 'USPS': 13, 'FDE': 0}, {'TLMD': 27, 'FDEG': 12, 'UPSN': 6, 'USPS': 5, 'FDE': 0}, {'TLMD': 38, 'FDEG': 16, 'UPSN': 8, 'USPS': 9, 'FDE': 0}, {'TLMD': 29, 'FDEG': 14, 'UPSN': 4, 'USPS': 2, 'FDE': 0}, {'TLMD': 25, 'FDEG': 14, 'UPSN': 12, 'USPS': 8, 'FDE': 0}, {'TLMD': 21, 'FDEG': 10, 'UPSN': 7, 'USPS': 4, 'FDE': 0}, {'TLMD': 26, 'FDEG': 16, 'UPSN': 13, 'USPS': 5, 'FDE': 0}, {'TLMD': 25, 'FDEG': 14, 'UPSN': 8, 'USPS': 12, 'FDE': 0}, {'TLMD': 23, 'FDEG': 13, 'UPSN': 12, 'USPS': 8, 'FDE': 0}, {'TLMD': 23, 'FDEG': 9, 'UPSN': 9, 'USPS': 10, 'FDE': 0}, {'TLMD': 32, 'FDEG': 13, 'UPSN': 10, 'USPS': 6, 'FDE': 0}, {'TLMD': 28, 'FDEG': 18, 'UPSN': 10, 'USPS': 6, 'FDE': 0}, {'TLMD': 32, 'FDEG': 14, 'UPSN': 4, 'USPS': 10, 'FDE': 1}]}, {'Truck Number': 2, 'pallets': [{'TLMD': 24, 'FDEG': 11, 'UPSN': 2, 'USPS': 7, 'FDE': 0}, {'TLMD': 26, 'FDEG': 15, 'UPSN': 7, 'USPS': 9, 'FDE': 0}, {'TLMD': 32, 'FDEG': 16, 'UPSN': 7, 'USPS': 10, 'FDE': 1

In [17]:
truck_data = assigned_packages
arrival_times_df = pd.DataFrame(df_package_distribution[['Truck Number', 'arrival_actualization']])
# Initialize lists to store data for DataFrame
pallet_numbers = []
package_numbers = []
arrival_times_list = []
scac_list = []

# Initialize package counter
package_counter = 1

# Initialize pallet counter
pallet_counter = 1

# Iterate over trucks and pallets to generate DataFrame data
for truck in truck_data:
    truck_number = truck['Truck Number']
    arrival_time = arrival_times_df[arrival_times_df['Truck Number'] == truck_number]['arrival_actualization'].values
    for pallet in truck['pallets']:
        scac_values = []
        for org, num_packages in pallet.items():
            scac_values.extend([org] * num_packages)
        np.random.shuffle(scac_values)  # Shuffle SCAC values within the pallet
        for scac in scac_values:
            pallet_numbers.append(pallet_counter)
            package_numbers.append(f"PKG{package_counter:06d}")
            arrival_times_list.append(arrival_time)
            scac_list.append(scac)
            package_counter += 1
        pallet_counter += 1
               

# Create DataFrame
df = pd.DataFrame({
    'pkg_received_utc_ts': arrival_times_list,
    'package_tracking_number': package_numbers,
    'scac': scac_list,
    'Pallet': pallet_numbers
})

print(df)

        

       pkg_received_utc_ts package_tracking_number  scac  Pallet
0      [51.14671738261082]               PKG000001  TLMD       1
1      [51.14671738261082]               PKG000002  TLMD       1
2      [51.14671738261082]               PKG000003  TLMD       1
3      [51.14671738261082]               PKG000004  TLMD       1
4      [51.14671738261082]               PKG000005  FDEG       1
...                    ...                     ...   ...     ...
10038  [939.0589881352732]               PKG010039  FDEG     175
10039  [939.0589881352732]               PKG010040  USPS     175
10040  [939.0589881352732]               PKG010041  FDEG     175
10041  [939.0589881352732]               PKG010042  FDEG     175
10042  [939.0589881352732]               PKG010043  FDEG     175

[10043 rows x 4 columns]
